In [1]:
import pandas as pd
import numpy as np

import spacy
import spacy_lookups_data

In [11]:
#Recommender module
from surprise import SVD, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering
from surprise.model_selection import cross_validate, LeaveOneOut
from surprise import accuracy
from surprise import Dataset, Reader
from surprise.model_selection import KFold

In [2]:
import fse
import recmetrics

ModuleNotFoundError: No module named 'fse'

In [17]:
reviews = pd.read_csv('../data/reviews_filtered.csv')
reviews.drop_duplicates(inplace=True)
print(reviews.shape)
reviews.head()

(320528, 5)


,name,review_stars,cleaned_text,sent_rating,uid
0,Deagan's Kitchen & Bar,5,we walked into melt did you want to put your n...,3.505159,0
1,Deagan's Kitchen & Bar,4,brunch on saturday was excellent the bloody ma...,3.800000,1
2,Deagan's Kitchen & Bar,4,great food great atmosphere great service some...,3.587500,2
3,Deagan's Kitchen & Bar,3,had a saturday evening dinner with friends goi...,3.436378,3
4,Deagan's Kitchen & Bar,2,i haven't been here for years i'm not from thi...,2.981408,4


In [ ]:
from fse import IndexedList

sentences = reviews['cleaned_text'].apply(lambda x: x.split(' ')).to_list()

In [7]:
import gensim.downloader as api
glove = api.load("glove-wiki-gigaword-100")

from fse.models import uSIF
model = uSIF(glove, workers=2, lang_freq="en")

In [8]:
from fse import IndexedList

s = IndexedList(sentences)

model.train(s)

(320528, 48666382)

In [9]:
matrix = []
for i in range(reviews.shape[0]):
    matrix.append(model.sv[i])
vector_df = pd.DataFrame(matrix)
del matrix

In [10]:
vector_df.to_csv('../data/vector_df.csv')

In [8]:
pd.read_csv(../data/vector:)

In [ ]:
## Recommendations from models

In [80]:
# Function to get the index of all comments

def get_all_comments(df, threshold=3):
    '''
    For all users, get the matrix with a list of index of all comments filtered by a certain rating
    '''
    uids = []
    idx = []
    df = df.sort_values('rating', ascending=False)
    for i in df.uid.unique():
        idx.append(df[(df.uid == i) & (df.rating > threshold)].index.to_list())
        uids.append(i)
        
    return pd.DataFrame({'user_id': uids, 'idx': idx})

In [ ]:
Int64Index([     0,   2428,   8065,  10903,  13433,  13833,  15173,  16110,
             17128,  24215,  39906,  40615,  53637,  57921,  63782,  70699,
             79369,  83957,  84338,  86324,  92158,  97228, 107436, 108632,
            113418, 116081, 117938, 119286, 134602, 137678, 140510, 163625,
            168947, 169198, 185520, 193467, 195405, 200461, 202727, 205928,
            206982, 230215, 234127, 246786, 259553, 262494, 262985, 266307,
            268634, 279347, 285612, 298029, 306375],

In [7]:
all_comments = get_all_comments(reviews)
all_comments

,user_id,idx
0,0,"[0, 2428, 8065, 10903, 13433, 13833, 15173, 16..."
1,1,"[1, 1153, 3958, 7358, 13369, 13756, 14772, 147..."
2,2,"[2, 15851, 33904, 34734, 38284, 39552, 40838, ..."
3,3,"[3, 1654, 2489, 14835, 15303, 15311, 18960, 18..."
4,4,"[4, 10549, 14080, 20495, 22484, 25038, 26466, ..."
...,...,...
4001,4001,"[39116, 41301, 42103, 44477, 46139, 50784, 518..."
4002,4002,"[39943, 55529, 59850, 61097, 66834, 81115, 879..."
4003,4003,"[42095, 47954, 54457, 58215, 58758, 58923, 594..."
4004,4004,"[45104, 46182, 46265, 48649, 67464, 67466, 795..."


In [82]:
rev = reviews.drop('sent_rating', axis=1)
rev.columns = ['name', 'rating', 'text', 'uid']
test = get_all_comments(rev)

In [85]:
test['actual'] = test['idx'].apply(get_restaurant_name)
test

NameError: name 'df' is not defined

In [106]:
# Function to given a list of comments, get the most similar excluding the comments in the list

def similar_comments(lst, k=10):
    
    idx_comments = []
    for i in lst:
        try:
            similar = [idx for idx in model.sv.most_similar(i)]
            for idx in similar:
                if idx not in idx_comments and idx[0] not in lst:
                    idx_comments.append(idx)
        except KeyError: # The index may has been eliminated
            continue
    # Filter by the similarity
    idx_comments.sort(key=lambda x:x[1], reverse=True)
    # Get only the k indices of comments
    result = [i[0] for i in idx_comments][:k]
    return result

In [ ]:
[i[0] for i in model.sv.most_similar(260127)]

In [ ]:
similar_comments([i[0] for i in model.sv.most_similar(260127)], 10)

In [9]:
all_comments['recommendations'] = all_comments['idx'].apply(similar_comments, k=10)

NameError: name 'model' is not defined

In [ ]:
all_comments.loc[0, 'recommendations']

In [73]:
### Functions for ratings and sentiment analysis

#Get recommendations based on rating:

def get_users_predictions(uid, df, k=10):
    recommended_items = df[['uid', 'rating']][df['uid'] == user_id]
    recommended_items = recommended_items.sort_values('rating', ascending=False).head(k)  
    return recommended_items.tolist()


In [75]:
test = rev.loc[0:2,'idx'].apply(similar_comments, k=10)
test

KeyError: 'idx'

In [101]:
from collections import OrderedDict

def get_restaurant_name(lst, df=reviews):
    '''
    Given a list of idx return the restaurant name
    '''
    result = [reviews.loc[i,'name'] for i in lst]
    return list(OrderedDict.fromkeys(result))

In [103]:
test['actual_rest'] = test['idx'].apply(get_restaurant_name)

,user_id,idx,actual_rest
0,0,"[0, 86324, 97228, 193467, 202727, 200461, 1179...","[Deagan's Kitchen & Bar, Noodlehead, Point Bru..."
1,3364,"[221796, 223264, 219084, 88614, 220118, 101854...","[Crumb & Spigot, Hook & Hoof New American Kitc..."
2,3758,"[87036, 221870, 221910, 218872, 218873, 218874...","[Paladar Latin Kitchen & Rum Bar, Ken Stewart'..."
3,679,"[221805, 220615, 219486, 88839, 81432, 208664,...","[Aces & Ales, Bottiglia Enoteca, Wolfgang Puck..."
4,1618,"[87032, 19734, 187599, 172053, 163425, 309358,...","[The Urban Tap, Primanti Bros, Meat & Potatoes..."


In [107]:
test['idx_pred'] = test['idx'].apply(similar_comments)

NameError: name 'model' is not defined

In [ ]:
test.head()

In [33]:
popular_rec = reviews.name.value_counts().head(10).index.to_list()

pop_recs = []
for user in reviews.index:
    pop_recs.append(popular_rec)
    
reviews['pop_predictions'] = pop_recs

In [37]:
# Recall at k (MarK)
recall_k_reviews = []
for K in np.arange(1,11):
    recall_k_reviews.extend([recmetrics.mark(popular_rec, popular_rec, k=K)])
recall_k_reviews

[0.06976260298840944,
 0.1395252059768189,
 0.2092878089652283,
 0.25920914211236784,
 0.32214634827538047,
 0.3835804279352666,
 0.44555494599273404,
 0.4954347928483873,
 0.5373506746567422,
 0.5737582992901734]

In [42]:
# Catalog

catalog = reviews.name.unique()
pop_coverage = recmetrics.coverage(pop_recs, catalog)

pop_coverage

0.07

In [44]:
# Intralist similarity

personalization = recmetrics.personalization(pop_recs)
personalization

KeyboardInterrupt: 

In [46]:
## Novelty
'''
nov = dict(reviews.name.value_counts())
pop_novelty, pop_mselfinfo_list = recmetrics.novelty(pop_recs, nov, reviews.uid.nunique(), 10)
print(pop_novelty, pop_mselfinfo_list)
'''

AttributeError: module 'recmetrics' has no attribute 'novelty'

In [47]:
help(recmetrics.plots)

Help on module recmetrics.plots in recmetrics:

NAME
    recmetrics.plots

FUNCTIONS
    class_separation_plot(pred_df, n_bins=150, threshold=None, figsize=(10, 6), title=None)
        Plots the predicted class probabilities for multiple classes.
        Usefull for visualizing predicted interaction values such as 5 star ratings, where a "class" is a star rating,
        or visualizing predicted class probabilities for binary classification model or recommender system.
        The true class states are colored.
        ----------
        pred_df: pandas dataframe
            a dataframe containing a column of predicted interaction values or classification probabilites,
            and a column of true class 1 and class 0 states.
            This dataframe must contain columns named "predicted" and "truth"
            example:
                    predicted | truth
                    5.345345        |  5
                    2.072020        |  2
        n_bins: number of bins for histogr

In [27]:
popular_rec

['Chipotle Mexican Grill',
 'The Cheesecake Factory',
 'Buffalo Wild Wings',
 'Yard House',
 'Red Robin Gourmet Burgers and Brews',
 'Olive Garden Italian Restaurant',
 "Oregano's Pizza Bistro",
 'Pita Jungle',
 'Taco Bell',
 "Denny's"]

In [3]:
#### Precision Recall at K
from collections import defaultdict

def apk(actual, pred, k=10, threshold=4):
    '''
    This functions calculates the precision and recall at K metric for each user in the dataset.
    '''   
    
    apks = []
    
    for a,p in (zip(actual, pred)):
        if len(pred) > k:
            pred = pred[:k]
        
        score = 0
        n_hits = 0
    
        for i, p in enumerate(pred):
            if p in actual and p not in predicted[:i]:
                n_hits +=1
                score += n_hits / (i+1)
    
        if not actual:
            apks.append(0)
        
        apks.append(score / min(len(actual), k))
        
        return np.mean(apks)
    
    

In [27]:
data = Dataset.load_from_df(df_sentiment, Reader(rating_scale=(1,5)))

In [30]:
def precision_recall_k(predictions, k=10, threshold=3.5):
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls

In [33]:
kf = KFold(n_splits=5)
svd = SVD()
for train, test in kf.split(data):
    svd.fit(train)
    predictions = svd.test(test)
    
    pred, rec = precision_recall_k(predictions, k=10)
    
    # Precision and recall can then be averaged over all users
    print(sum(prec for prec in pred.values()) / len(pred))
    print(sum(rec for rec in rec.values()) / len(rec))

KeyboardInterrupt: 

In [24]:
df_sentiment = reviews[['uid', 'name', 'sent_rating']]
df_sentiment.columns = ['uid', 'name', 'rating']

def get_users_predictions(uid, df, k=10):
    recommended_items = df[['uid', 'rating']][df['uid'] == uid]
    recommended_items = recommended_items.sort_values('rating', ascending=False).head(k)  
    return recommended_items

In [25]:
get_users_predictions(0, df_sentiment)

,uid,rating
24215,0,3.729104
16110,0,3.695986
107436,0,3.624653
10903,0,3.612542
169198,0,3.607481
262494,0,3.570370
97228,0,3.555586
39906,0,3.552323
259553,0,3.550278
113418,0,3.541444


In [ ]:
pred = 